In [8]:
import numpy as np
import math
# paramters
L1 = 0.3
L2 = 0.3

x_o=0.5
y_o=0.0
# target position
x_g = 0.1
y_g = 0.3

#Forward kinematic function
def forward_kinematics(theta1, theta2):
    x = x_o+L1 * np.cos(theta1) + L2 * np.cos(theta1 + theta2)
    y = y_o+L1 * np.sin(theta1) + L2 * np.sin(theta1 + theta2)
    return np.array([x, y])

# jacobian function
def jacobian(theta1, theta2):
    J11 = -L1 * np.sin(theta1) - L2 * np.sin(theta1 + theta2)
    J12 = -L2 * np.sin(theta1 + theta2)
    J21 = L1 * np.cos(theta1) + L2 * np.cos(theta1 + theta2)
    J22 = L2 * np.cos(theta1 + theta2)
    return np.array([[J11, J12], [J21, J22]])

# inverse process 
def inverse_kinematics(target, initial_guess, tol=1e-6, max_iter=100):
    theta = np.array(initial_guess, dtype=float)
    for _ in range(max_iter):
        current_pos = forward_kinematics(theta[0], theta[1])
        error = target - current_pos
        if np.linalg.norm(error) < tol:
            break
        J = jacobian(theta[0], theta[1])
        print("J:",J)
        dtheta = np.linalg.pinv(J) @ error
        theta += dtheta
    return theta

# initial iteration angle
initial_guess = [0.0, 0.0]

# target position
target = np.array([x_g, y_g])

# calculation the angles via the kinematic
theta1, theta2 = inverse_kinematics(target, initial_guess)

# validate the result
x, y = forward_kinematics(theta1, theta2)
# get the degrees
theta1_deg = math.degrees(theta1)
theta2_deg = math.degrees(theta2)

print(f"angle info(radians): theta1={theta1}, theta2={theta2}")
print(f"angle info(degrees): theta1={theta1_deg%360}, theta2={theta2_deg%360}")
print(f"the end position: x={x}, y={y}")


J: [[-0.  -0. ]
 [ 0.6  0.3]]
J: [[-0.28621824 -0.16939274]
 [ 0.52391898  0.24760068]]
J: [[-0.32811145 -0.23866549]
 [ 0.10458967 -0.18176574]]
J: [[-0.07078968  0.21887638]
 [-0.12710476 -0.2051661 ]]
J: [[-0.37103053 -0.07340068]
 [-0.25324597 -0.290882  ]]
J: [[-0.27177627  0.01537887]
 [-0.38644983 -0.29960556]]
J: [[-0.29978068 -0.01647562]
 [-0.39822978 -0.29954725]]
J: [[-0.29999411 -0.01732781]
 [-0.39999675 -0.29949916]]
angle info(radians): theta1=-10.653964613368714, theta2=26.304112316194917
angle info(degrees): theta1=109.5727925722448, theta2=67.11461955605705
the end position: x=0.10000000009492283, y=0.29999999997116067


In [10]:
import numpy as np
import math

# Parameters
L1 = 0.3
L2 = 0.3

x_o = 0.5
y_o = 0.0
# Target position
x_g = 0.1
y_g = 0.3

# Forward kinematic function
def forward_kinematics(theta1, theta2):
    x = x_o + L1 * math.cos(theta1) + L2 * math.cos(theta1 + theta2)
    y = y_o + L1 * math.sin(theta1) + L2 * math.sin(theta1 + theta2)
    return np.array([x, y])

# Jacobian function
def jacobian(theta1, theta2):
    J11 = -L1 * math.sin(theta1) - L2 * math.sin(theta1 + theta2)
    J12 = -L2 * math.sin(theta1 + theta2)
    J21 = L1 * math.cos(theta1) + L2 * math.cos(theta1 + theta2)
    J22 = L2 * math.cos(theta1 + theta2)
    return np.array([[J11, J12], [J21, J22]])

# Function to calculate the pseudoinverse of a 2x2 matrix
# Function to calculate the pseudoinverse of a 2x2 matrix manually
def pseudo_inverse(J):
    print("J:",J)
    det = J[0,0] * J[1,1] - J[0,1] * J[1,0]
    if abs(det) < 1e-6:
        # Handle singular matrix by returning a least-squares solution
        # Here we add a small value to the diagonal (Tikhonov regularization)
        J[0,0] += 1e-6
        J[1,1] += 1e-6
        det = J[0,0] * J[1,1] - J[0,1] * J[1,0]
    inv_det = 1.0 / det
    J_inv = np.array([[J[1,1] * inv_det, -J[0,1] * inv_det],
                      [-J[1,0] * inv_det, J[0,0] * inv_det]])
    return J_inv

# Inverse kinematics process 
def inverse_kinematics(target, initial_guess, tol=1e-6, max_iter=100):
    theta = np.array(initial_guess, dtype=float)
    for _ in range(max_iter):
        current_pos = forward_kinematics(theta[0], theta[1])
        error = target - current_pos
        if np.linalg.norm(error) < tol:
            break
        J = jacobian(theta[0], theta[1])
        J_inv = pseudo_inverse(J)
        dtheta = np.dot(J_inv, error)
        theta += dtheta
    return theta

# Initial iteration angle
initial_guess = [0.0, 0.0]

# Target position
target = np.array([x_g, y_g])

# Calculation the angles via the kinematic
theta1, theta2 = inverse_kinematics(target, initial_guess)

# Validate the result
x, y = forward_kinematics(theta1, theta2)
# Get the degrees
theta1_deg = math.degrees(theta1)
theta2_deg = math.degrees(theta2)

print(f"angle info(radians): theta1={theta1}, theta2={theta2}")
print(f"angle info(degrees): theta1={theta1_deg % 360}, theta2={theta2_deg % 360}")
print(f"the end position: x={x}, y={y}")


J: [[-0.  -0. ]
 [ 0.6  0.3]]
J: [[-0.18182454 -0.07682649]
 [ 0.57102166  0.28999602]]
J: [[-0.14155347 -0.22920329]
 [ 0.0933493  -0.19356098]]
J: [[ 0.01891945  0.22049597]
 [-0.01876215  0.2034245 ]]
J: [[ 0.05951341  0.17762023]
 [-0.03400653  0.24176653]]
J: [[-0.00604071  0.29394983]
 [ 0.06232867  0.0599458 ]]
J: [[-0.25024481  0.04475176]
 [-0.24208123 -0.29664336]]
J: [[-0.25914408  0.02351954]
 [-0.39958173 -0.29907663]]
J: [[-0.30007764 -0.01646045]
 [-0.39732993 -0.29954808]]
J: [[-0.29998693 -0.01731957]
 [-0.39999424 -0.29949964]]
angle info(radians): theta1=-999985.8787875529, theta2=1999932.7714610444
angle info(degrees): theta1=109.57279250025749, theta2=67.11461965739727
the end position: x=0.1000000004248448, y=0.2999999999230725
